# Zero-shot mutant prediction for Cas12a

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://l0:7890"
os.environ["https_proxy"] = "http://l0:7890"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/cas12a.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [7]:
base_model_path = "AI4Protein/cas12a_model_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

  0%|          | 0/23332 [00:00<?, ?it/s]

Model 0 done


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

...

Model 3 done


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

  0%|          | 0/23332 [00:00<?, ?it/s]

Model 4 done


In [9]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [ ]:
df